In [32]:
from datasets import load_dataset
from datasets import Dataset, DatasetDict
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelWithLMHead, TranslationPipeline

import torch
import pandas as pd

### Load the dataset

In [33]:
articles = pd.read_csv('/Users/Blanche/Downloads/CONSTITUANTE_Avant-projet après 1ère lecture (décembre 2021).csv')
articles.columns = ['article_text', 'artId']
val1 = articles['artId'].dropna().reset_index()
val2 = articles['article_text'].dropna().reset_index()
articles = pd.concat([val2, val1], axis=1)
articles.pop("index")
print(articles.head())

                                        article_text   artId
0  République et Canton du Valais Le canton du Va...  Art. 1
1  Organisation du Canton Le canton du Valais est...  Art. 2
2  Capitale Sion est la capitale du canton du Val...  Art. 3
3  Armoiries Les armoiries sont : Parti d'argent ...  Art. 4
4  Hymne valaisan L’hymne valaisan officiel est c...  Art. 5


In [34]:
articles_train = Dataset.from_pandas(articles)

ds = DatasetDict()

ds['train'] = articles_train

print(ds)

DatasetDict({
    train: Dataset({
        features: ['article_text', 'artId'],
        num_rows: 206
    })
})


### Create a test and a train set for translation 

In [38]:
ds = ds["train"].train_test_split(test_size=0.2)

In [39]:
ds["train"][0]

{'article_text': 'Relations extérieures Le Conseil d’État représente le canton. Il négocie et signe les accords intercantonaux et transfrontaliers, sous réserve des compétences du Grand  Conseil. Il informe régulièrement ce dernier des négociations en cours. Il répond aux consultations fédérales. Le Conseil d’État et la députation valaisanne aux Chambres fédérales constituent, selon les modalités fixées  par la loi, une commission permanente d’échange d’informations relatives aux affaires fédérales, dénommée  Conférence des affaires fédérales.',
 'artId': 'Art. 89'}

### Initialize the models and the tokenizers for translation and classification

In [35]:
model_name_translation = "t5-small"
model_name_classification = "MoritzLaurer/policy-distilbert-7d"

model_classification = AutoModelForSequenceClassification.from_pretrained(model_name_classification)
tokenizer_classification  = AutoTokenizer.from_pretrained(model_name_classification)

model_translation = AutoModelForSeq2SeqLM.from_pretrained(model_name_translation)
tokenizer_translation = AutoTokenizer.from_pretrained(model_name_translation)

Downloading: 100%|█████████████████████████| 1.20k/1.20k [00:00<00:00, 99.0kB/s]
Downloading: 100%|███████████████████████████| 242M/242M [00:27<00:00, 8.71MB/s]
Downloading: 100%|███████████████████████████| 792k/792k [00:00<00:00, 1.16MB/s]
Downloading: 100%|█████████████████████████| 1.39M/1.39M [00:01<00:00, 1.19MB/s]
/usr/local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate thi

### Tokenize the dataset


### Train the translation process